# HuggingTweets - Tweet Generation with Huggingface

*Disclaimer: this project is not to be used to publish any false generated information but to perform research on Natural Language Generation (NLG).*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets.ipynb)

## Introduction

Generating realistic text has become more and more efficient with models such as [GPT-2](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf). Those models are trained on very large datasets and require heavy computer resources (and time!).

However, we can use Transfer Learning and a single GPU to quickly fine-tune a pre-trained model on a given task.

We test if we can imitate the writing style of a Twitter user by only using some of his tweets. Twitter API let us download "only" the 3200 most recent tweets from any single user, which we then filter out (to remove retweets, short content, etc).

[HuggingFace](https://huggingface.co/) gives us an easy access to pre-trained models and fine-tuning techniques for Natural Language Generation (NLG) tasks.

We will be monitoring the training with [W&B](https://docs.wandb.com/huggingface) (which is integrated in HuggingFace) to ensure the model is learning from the data and compare multiple experiments.

![](https://i.imgur.com/vnejHGh.png)

## Install dependencies

In [ ]:
# install required libraries are not installed
!pip install torch -qq
!pip install transformers -qq
!pip install wandb -qq
!pip install tweepy -qq

In [ ]:
# HuggingFace scripts for fine-tuning models and language generation
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py -q
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/text-generation/run_generation.py -q

## Set up a Twitter Development Account

In order to access Twitter data, we need to:

* [create a Twitter development account](https://developer.twitter.com/en/apply-for-access)
* [create a Twitter app](https://developer.twitter.com/en/apps)
* get your consumer API keys: "API key" and "API secret key"

The entire process only takes a few minutes.

In [ ]:
# <--- Enter your credentials (don't share with anyone) --->
consumer_key = 'XXXXXXXX'
consumer_secret = 'XXXXXXXX'

## Download tweets from a user

We download latest tweets associated to a user account through [Tweepy](http://docs.tweepy.org/).

In [ ]:
import tweepy

In [ ]:
# authenticate
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

We grab all available tweets (limited to 3200 per API limitations) based on Twitter handle.

**Note**: Protected users may only be requested when the authenticated user either "owns" the timeline or is an approved follower of the owner.

In [ ]:
# <--- Enter the screen name of the user you will download your dataset from --->
handle = 'elonmusk'

In [ ]:
# Adapted from https://gist.github.com/onmyeoin/62c72a7d61fc840b2689b2cf106f583c

# initialize a list to hold all the tweepy Tweets & list with no retweets
alltweets = []
n_requests = 0

# make initial request for most recent tweets with extended mode enabled to get full tweets
for _ in range(3):
    new_tweets = api.user_timeline(
        screen_name=handle, tweet_mode='extended', count=200)
    n_requests += 1
    if new_tweets: break

if new_tweets:
    # save most recent tweets
    alltweets.extend(new_tweets)

    # save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    # check we cannot get more tweets
    no_tweet_count = 0

    # keep grabbing tweets until the api limit is reached
    while True:
        # all subsequent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(
            screen_name=handle, tweet_mode='extended', count=200, max_id=oldest)
        n_requests += 1
        
        # stop if no more tweets (try a few times as they sometimes eventually come)
        if not new_tweets:
            no_tweet_count +=1
        else:
            no_tweet_count = 0
        if no_tweet_count > 5: break

        # save most recent tweets
        alltweets.extend(new_tweets)

        # update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%i tweets downloaded so far" %
              (len(alltweets)))

n_tweets = len(alltweets)

print("Grabbed %i tweets after %i requests and %i retries" %
      (n_tweets, n_requests, no_tweet_count))

In [ ]:
# get text and remove RT
my_tweets = [tweet.full_text for tweet in alltweets if not hasattr(tweet, 'retweeted_status')]

print("Found %i tweets, including %i RT, keeping %i" %
      (n_tweets, n_tweets - len(my_tweets), len(my_tweets)))

print("Rate limit: ", api.rate_limit_status()['resources']['statuses']['/statuses/user_timeline'])

## Create a dataset from downloaded tweets

We remove:
* retweets (since it's not in the wording style of target author)
* tweets with no interesting content (limited to url's, user mentionss, "thank you"…)

We clean up remaining tweets:
* we remove url's
* we replace "@" mentions with user names

In [ ]:
import random
import re
import torch

We verify our list of tweets is well curated.

In [ ]:
print(f'Total number of tweets: {len(alltweets)}\nMy tweets: {len(my_tweets)}')

In [ ]:
print('Original tweets\n')
for t in alltweets[:5]:
    print(f'{t.full_text}\n')

In [ ]:
print('My tweets\n')
for t in my_tweets[:5]:
    print(f'{t}\n')

We remove boring tweets (tweets with only urls or too short) and cleanup texts.

In [ ]:
def fix_text(text):
    text = text.replace('&amp;', '&')
    text = text.replace('&lt;', '<')
    text = text.replace('&gt;', '>')
    return text

In [ ]:
def clean_tweet(tweet, allow_new_lines = False):
    bad_start = ['http:', 'https:']
    for w in bad_start:
        tweet = re.sub(f" {w}\\S+", "", tweet)      # removes white space before url
        tweet = re.sub(f"{w}\\S+ ", "", tweet)      # in case a tweet starts with a url
        tweet = re.sub(f"\n{w}\\S+ ", "", tweet)    # in case the url is on a new line
        tweet = re.sub(f"\n{w}\\S+", "", tweet)     # in case the url is alone on a new line
        tweet = re.sub(f"{w}\\S+", "", tweet)       # any other case?
    tweet = re.sub(' +', ' ', tweet)                # replace multiple spaces with one space
    if not allow_new_lines:                         # TODO: predictions seem better without new lines
        tweet = ' '.join(tweet.split())
    return tweet.strip()

In [ ]:
def boring_tweet(tweet):
    "Check if this is a boring tweet"
    boring_stuff = ['http', '@', '#']
    not_boring_words = len([None for w in tweet.split() if all(bs not in w.lower() for bs in boring_stuff)])
    return not_boring_words < 3

In [ ]:
curated_tweets = [fix_text(tweet) for tweet in my_tweets]
clean_tweets = [clean_tweet(tweet) for tweet in curated_tweets]
cool_tweets = [tweet for tweet in clean_tweets if not boring_tweet(tweet)]

In [ ]:
print(f'Curated tweets: {len(curated_tweets)}\nCool tweets: {len(cool_tweets)}')

We split data into training and validation sets (90/10).

In [ ]:
# shuffle data
random.shuffle(cool_tweets)

# fraction of training data
split_train_valid = 0.9

# split dataset
train_size = int(split_train_valid * len(cool_tweets))
valid_size = len(cool_tweets) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(cool_tweets, [train_size, valid_size])

We export our datasets as text files, simulating number of epochs by mixing up tweets, due to one batch containing multiple tweets.

In [ ]:
def make_dataset(dataset, epochs):
    total_text = '<|endoftext|>'
    tweets = [t for t in dataset]
    for _ in range(epochs):
        random.shuffle(tweets)
        total_text += '<|endoftext|>'.join(tweets) + '<|endoftext|>'
    return total_text

In [ ]:
EPOCHS = 4

with open('{}_train.txt'.format(handle), 'w') as f:
    data = make_dataset(train_dataset, EPOCHS)
    f.write(data)

with open('{}_valid.txt'.format(handle), 'w') as f:
    data = make_dataset(valid_dataset, 1)
    f.write(data)

## Log and monitor training through W&B

In order to check our model is training correctly and compare experiments, we are going to use the W&B integration from HuggingFace.

### API Key
Once you've signed up, run the next cell and click on the link to get your API key and authenticate this notebook.

In [ ]:
import wandb
wandb.login()

## Fine-tuning the model

HuggingFace includes the script `run_language_modeling` making it easy to fine-tune a pre-trained model.

We use a pre-trained GPT-2 model and fine-tune it on our dataset.

Training is automatically logged on W&B (see [documentation](https://docs.wandb.com/huggingface)). Urls are generated to visualize ongoing runs or you can just open your [dashboard](http://app.wandb.ai/).

I quickly tested running for several epochs and my run was showing I started overfitting after 4 epochs so this is the limit I use to fine-tune my model (takes less than 2 minutes).

![](https://i.imgur.com/1uIxLFe.png)

In [ ]:
# Associate run to a project (optional)
%env WANDB_PROJECT=huggingtweets-dev

We use HuggingFace script `run_language_modeling.py` to fine-tune our model (see [doc](https://huggingface.co/transformers/)).

*Note: epochs are built into the dataset*

In [ ]:
!python run_language_modeling.py \
    --output_dir=output/$handle \
    --overwrite_output_dir \
    --overwrite_cache \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train --train_data_file=$handle\_train.txt \
    --do_eval --eval_data_file=$handle\_valid.txt \
    --evaluate_during_training \
    --eval_steps 20 \
    --logging_steps 20 \
    --per_gpu_train_batch_size 1 \
    --num_train_epochs 1

## Let's test our trained model!

We test our model on a few sample sentences.

In [ ]:
SENTENCES = ["I think that",
             "I like",
             "I don't like",
             "I want",
             "My dream is"]

We use HuggingFace script `run_generation.py` to generate sentences (see [doc](https://huggingface.co/transformers/)).

In [ ]:
import random
seed = random.randint(0, 2**32-1)
seed

In [ ]:
examples = []
num_return_sequences = 5

for start in SENTENCES:
    val = !python run_generation.py \
        --model_type gpt2 \
        --model_name_or_path output/$handle \
        --length 160 \
        --num_return_sequences $num_return_sequences \
        --temperature 1 \
        --p 0.95 \
        --seed $seed \
        --prompt {'"<|endoftext|>' + start + '"'}
    generated = [val[-1-2*k] for k in range(num_return_sequences)[::-1]]
    print(f'\nStart of sentence: {start}')
    for i, g in enumerate(generated):
        g = g.replace('<|endoftext|>', '')
        print(f'* Generated #{i+1}: {g}')

## About

*Built by Boris Dayma*

[![Follow](https://img.shields.io/twitter/follow/borisdayma?style=social)](https://twitter.com/borisdayma)

My main goals with this project are:
* to experiment with how to train, deploy and maintain neural networks in production ;
* to make AI accessible to everyone ;
* to have fun!

For more details, visit the project repository.

[![GitHub stars](https://img.shields.io/github/stars/borisdayma/huggingtweets?style=social)](https://github.com/borisdayma/huggingtweets)

**Disclaimer: this project is not to be used to publish any false generated information but to perform research on Natural Language Generation.**

## Resources

* [Explore the W&B report](https://bit.ly/2TGXMZf) to understand how the model works
* [HuggingFace and W&B integration documentation](https://docs.wandb.com/library/integrations/huggingface)

## Got questions about W&B?

If you have any questions about using W&B to track your model performance and predictions, please reach out to the [slack community](http://bit.ly/wandb-forum).